### Student and Examinations

In [3]:
import pandas as pd
import pandasql as ps

data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
Students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
Subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
Examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

print(Students, Subjects, Examinations, sep = '\n\n')


   student_id student_name
0           1        Alice
1           2          Bob
2          13         John
3           6         Alex

  subject_name
0         Math
1      Physics
2  Programming

    student_id subject_name
0            1         Math
1            1      Physics
2            1  Programming
3            2  Programming
4            1      Physics
5            1         Math
6           13         Math
7           13  Programming
8           13      Physics
9            2         Math
10           1         Math


In [36]:
all_value = pd.merge(Students, Subjects, how = 'cross')
print(all_value)

    student_id student_name subject_name
0            1        Alice         Math
1            1        Alice      Physics
2            1        Alice  Programming
3            2          Bob         Math
4            2          Bob      Physics
5            2          Bob  Programming
6           13         John         Math
7           13         John      Physics
8           13         John  Programming
9            6         Alex         Math
10           6         Alex      Physics
11           6         Alex  Programming


In [79]:
results = pd.merge(all_value, Examinations, on = ['student_id', 'subject_name'], how = 'left', indicator=True)
results.groupby(['student_id','student_name','subject_name']).apply(lambda x : (x['_merge'] == 'both').sum()).reset_index(name = 'attended_count').sort_values(by=['student_id','subject_name'])

,student_id,student_name,subject_name,attended_count
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Physics,0
5,2,Bob,Programming,1
6,6,Alex,Math,0
7,6,Alex,Physics,0
8,6,Alex,Programming,0
9,13,John,Math,1


In [66]:
query = '''
with ct as(
    select
        S.student_id,
        S.student_name,
        Sub.subject_name
    from Students S
    cross join Subjects Sub
    )

select
    ct.student_id,
    ct.student_name,
    ct.subject_name,
    count(Exam.subject_name) as attended_exams
from ct
left join Examinations Exam
on ct.student_id = Exam.student_id and ct.subject_name = Exam.subject_name
group by ct.student_id, ct.student_name, ct.subject_name
order by ct.student_id, ct.subject_name
'''

ps.sqldf(query, locals())

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Physics,0
5,2,Bob,Programming,1
6,6,Alex,Math,0
7,6,Alex,Physics,0
8,6,Alex,Programming,0
9,13,John,Math,1


### Managers with at least 5 Direct Reports

In [43]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
Employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

print(Employee)

    id   name department  managerId
0  101   John          A       <NA>
1  102    Dan          A        101
2  103  James          A        101
3  104    Amy          A        101
4  105   Anne          A        101
5  106    Ron          B        101


In [55]:
results = Employee.groupby('managerId').count().reset_index()
print(pd.merge(results, Employee, left_on = 'managerId', right_on='id',how = 'left', suffixes=('_left','_right'))[['name_right']].rename(columns={'name_right':'name'}))

   name
0  John


In [108]:
query = '''
select e.name
from Employee e
where e.id in (
    select managerId
    from Employee
    group by managerId
    having count(*) >= 5
)
'''
print(ps.sqldf(query, locals()))

   name
0  John


In [57]:
# self join

query= '''
select distinct m.name
from Employee e
join Employee m on e.managerId = m.id
group by m.id, m.name
having count(e.id) >= 5;
'''

print(ps.sqldf(query, locals()))

   name
0  John


### Sales Person

In [4]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
SalesPerson = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
Company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
Orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})

print(SalesPerson, Company, Orders, sep = '\n\n')

   sales_id  name  salary  commission_rate  hire_date
0         1  John  100000                6 2006-04-01
1         2   Amy   12000                5 2010-05-01
2         3  Mark   65000               12 2008-12-25
3         4   Pam   25000               25 2005-01-01
4         5  Alex    5000               10 2007-02-03

   com_id    name      city
0       1     RED    Boston
1       2  ORANGE  New York
2       3  YELLOW    Boston
3       4   GREEN    Austin

   order_id order_date  com_id  sales_id  amount
0         1 2014-01-01       3         4   10000
1         2 2014-02-01       4         5    5000
2         3 2014-03-01       1         1   50000
3         4 2014-04-01       1         4   25000


In [41]:
query = '''
select name
from SalesPerson
where sales_id not in(
  select distinct Orders.sales_id
  from Orders
  join Company on Orders.com_id = Company.com_id
  where Company.name = "RED"
)
'''

print(ps.sqldf(query, locals()))

   name
0   Amy
1  Mark
2  Alex


In [65]:
query = '''
select name
from SalesPerson sp
where not exists (
    select 1
    from Orders o
    join Company c on o.com_id = c.com_id
    where c.name = "RED" and o.sales_id = sp.sales_id
);
'''

print(ps.sqldf(query, locals()))

   name
0   Amy
1  Mark
2  Alex


In [5]:
left_df = pd.merge(Company, Orders, on = 'com_id')
left_df

,com_id,name,city,order_id,order_date,sales_id,amount
0,1,RED,Boston,3,2014-03-01,1,50000
1,1,RED,Boston,4,2014-04-01,4,25000
2,3,YELLOW,Boston,1,2014-01-01,4,10000
3,4,GREEN,Austin,2,2014-02-01,5,5000


In [6]:
result = pd.merge(SalesPerson, left_df, on = 'sales_id', how = 'left', suffixes = ('_sales', '_company'))
result

,sales_id,name_sales,salary,commission_rate,hire_date,com_id,name_company,city,order_id,order_date,amount
0,1,John,100000,6,2006-04-01,1,RED,Boston,3,2014-03-01,50000
1,2,Amy,12000,5,2010-05-01,<NA>,NaN,NaN,<NA>,NaT,<NA>
2,3,Mark,65000,12,2008-12-25,<NA>,NaN,NaN,<NA>,NaT,<NA>
3,4,Pam,25000,25,2005-01-01,1,RED,Boston,4,2014-04-01,25000
4,4,Pam,25000,25,2005-01-01,3,YELLOW,Boston,1,2014-01-01,10000
5,5,Alex,5000,10,2007-02-03,4,GREEN,Austin,2,2014-02-01,5000


In [7]:
red_order = result[result['name_company']=='RED']['sales_id'].unique()
result[~result['sales_id'].isin(red_order)][['name_sales']]

,name_sales
1,Amy
2,Mark
5,Alex


In [60]:
# 최적화

red_orders = Orders[Orders['com_id'].isin(Company[Company['name'] == 'RED']['com_id'])]
red_sales_ids = red_orders['sales_id'].unique()
SalesPerson[~SalesPerson['sales_id'].isin(red_sales_ids)][['name']]

,order_id,order_date,com_id,sales_id,amount
2,3,2014-03-01,1,1,50000
3,4,2014-04-01,1,4,25000
